### Импорт библиотек

Алгоритм правильно обрабатывает визиты с одной позицией!

In [1]:
import numpy as np
import pandas as pd
import csv
from datetime import datetime

# Описание признаков purchase.csv

0) **client_id** - уникальный индетификатор клиента

1) **transaction_id** - уникальный индетификатор транзакции(покупки)

2) **transaction_datetime** - дата совершения покупки "2018-12-01 07:12:45" ('%Y-%m-%d %H:%M:%S')

3) **regular_points_received** - количество полученных "регулярных" баллов за покупку

4) **express_points_received** - количество полученных "экспресс" баллов за покупку

5) **regular_points_spent** - количество потраченных "регулярных" баллов за покупку

6) **express_points_spent** - количество потраченных "экспресс" баллов за покупку

7) **purchase_sum** - сумма совершенной покупки (все позиции)

8) **store_id** - уникальный индетификатор магазина

9) **product_id** - уникальный индетификатор продукта

10) **product_quantity** - количество данного продукта в корзине

11) **trn_sum_from_iss** - стоимость данной позиции из чека

12) **trn_sum_from_red** - стоимость данной позиции из чека без скидки

# Описание итоговых признаков в DataFrame

### Чистые признаки
0) **client_id**

1) **n_visits** - число визитов клиента в магазин

2) **n_red_visits** - число "красных" визитов клиента в магазин (со скидкой)

3) **weekday_1** - число посещений в понедельник

...

8) **weekday_7** - число посещений в воскресенье

### Сгенерированные признаки

К следующим 11 признакам применяется стандартные статистические параметры.

**Максимум / Мининимум / Среднее / Медиана / Дисперсия**


9) **delay_days** - дилэй между посещениями ( 0 - следующее посещение в течение суток )

...

14) **regular_points_received** - количество полученных "регулярных" баллов за покупку

...

19) **express_points_received** - количество полученных "экспресс" баллов за покупку

...

24) **regular_points_spent** - количество потраченных "регулярных" баллов за покупку

...

29) **express_points_spent** - количество потраченных "экспресс" баллов за покупку

...

34) **purchase_sum** - сумма совершенной покупки (все позиции)

...

39) **sum_product_quantity** - количество товаров в корзине ( весовой товар = 1 ) 

...

44) **sum_product_unique** - количество уникальных товаров в корзине

...

49) **trn_sum_from_iss** - цена одной позиции в чеке 

...

54) **trn_sum_from_red** - цена одной позиции в чеке оплаченной бонусами без скидки 

...

59) **price_from_iss** - цена одной единицы товара в позиции

...

In [2]:
PATH='Data'
filename = PATH + '/purchases.csv'

In [3]:
# Создаем генератор строк
def generator_of_rows(filename):
    '''
    Generator of rows in csv file
    Returns row per call.
    
    '''
    # Открываем файл
    with open(filename, "r") as csvfile:
        datareader = csv.reader(csvfile)
        for row in datareader:
            yield row

In [4]:
# Создаем список функций которые хотим применять, для извлечения признаков
list_of_functions = [np.max, np.min, np.mean, np.median, np.std]


def create_feature(list_func,lists_of_feature):
    '''
    Create features by applying functions to list of features.
    Takes list of functions and apply them to lists of given features.
    Returns list with new generated features.
    
    '''
    result_list=[]
    for feature in lists_of_feature:
        for func in list_func:
            result_list.append(func(feature))
    return result_list

def clear_lists(lists_of_feature):
    '''Delete lists '''
    for feature_list in lists_of_feature:
        feature_list=[]

In [5]:
def inizialize_row(id_of_row, csv_row, last_row = None):
    '''
    Assign values to current and previos row.
    Takes id_of_row and row to update current row and previos row.
    Returns current row and previos row.
    
    '''
    # При первом запуске last_row = None
    if id_of_row==1:
        prev_row = csv_row
        curr_row = csv_row
    
    # После этого last_row = previos_row
    if id_of_row > 1:
        prev_row = last_row
        curr_row = csv_row
    return curr_row, prev_row

In [6]:
def uptate_curr_check_feature(row, sum_product_unique,
                                sum_product_quantity, list_trn_sum_from_iss,
                                list_price_from_iss, list_trn_sum_from_red):
    
    '''
    Update info about current check due to next iteration.
    Takes arguments from last iteration and update it due to info in row.
    Returns updated features.
    
    '''
    #число уникальных товаров в чеке +1
    sum_product_unique += 1

    #общее число товаров в чеке
    if float(row[10])==0:
        quantity = 1.0
    else:
        quantity = float(row[10]) 
    sum_product_quantity += quantity

    #стоимость позиции в чеке
    trn_sum_from_iss = float(row[11])
    list_trn_sum_from_iss.append(trn_sum_from_iss)

    #стоимость товара в чеке
    price_from_iss = trn_sum_from_iss/quantity
    list_price_from_iss.append(price_from_iss)

    #стоимость позизии оплаченной бонусами в чеке
    if row[12]:
        trn_sum_from_red = float(row[12])
    else:
        trn_sum_from_red = 0 
    list_trn_sum_from_red.append(trn_sum_from_red)
    list_to_return = [sum_product_unique,
                      sum_product_quantity,
                      list_trn_sum_from_iss,
                      list_price_from_iss,
                      list_trn_sum_from_red
                     ]
    return list_to_return

In [7]:
def create_feat_from_days(client_visit_days):
    '''
    Uses list with dates of visits.
    Returns list with delays between visits in days.
    Also returns number of visits grouped by weekday. 

    '''
    weekday_list = [0,0,0,0,0,0,0]
    for date in client_visit_days:
        n_week_day = datetime.strptime(date, '%Y-%m-%d %H:%M:%S').weekday()
        weekday_list[n_week_day]+=1

    list_delay_days = []
    if len(client_visit_days)==1:

        # Если у клиента только одно посещение помечаем его -1
        list_delay_days.append(-1)
    else:
        # Вычисляем дилэй между ближайшими посещениями
        for i in range(len(client_visit_days)-1):
            date_0 = datetime.strptime(client_visit_days[i], '%Y-%m-%d %H:%M:%S')
            date_1 = datetime.strptime(client_visit_days[i+1], '%Y-%m-%d %H:%M:%S')
            delay = (date_1 - date_0).days

            list_delay_days.append(delay)




    return list_delay_days, weekday_list

In [8]:
def summarize_last_visit(previos_row, prev_date, list_of_last_visit):
    '''
    Returns updated list with info about last visit.
    Takes previos row, previos date and list_of_last_visit 
    where:
    0 list_client_day,\
    1 list_regular_points_received,\
    2 list_express_points_received,\
    3 list_regular_points_spent,\
    4 list_express_points_spent,\
    5 list_purchase_sum,\
    6 list_sum_product_quantity,\
    7 list_sum_product_unique,\
    8 sum_product_unique,\
    9 sum_product_quantity = list_of_last_visit
    
    '''
    # Подводим итоги прошлого визита!
    # Добавляем день посещения
    list_of_last_visit[0].append(prev_date)

    # Добавляем информацию о бонусах за посещение
    list_of_last_visit[1].append(float(previos_row[3]))
    list_of_last_visit[2].append(float(previos_row[4]))
    list_of_last_visit[3].append(float(previos_row[5]))
    list_of_last_visit[4].append(float(previos_row[6]))

    # Добавляем информацию о количестве и качестве покупок
    list_of_last_visit[5].append(float(previos_row[7]))
    list_of_last_visit[6].append(float(list_of_last_visit[9]))
    list_of_last_visit[7].append(float(list_of_last_visit[8]))
    list_of_last_visit[8] , list_of_last_visit[9] = 0, 0

    return list_of_last_visit

In [9]:
def create_data_for_df(nmax_rows, filename='Data/purchases.csv'):    
    '''
    This is the key function in purcing csv file.
    Running rows in given csv file and create list with final features.
    Returns lists with info about each person.
    '''
    id_of_row = 0
    purchase=[]
    n_of_user = 0
    #filename = 'Data/purchases.csv'
    for row in generator_of_rows(filename):
        # пропускаем header
        if id_of_row==0:
            id_of_row = 1
            continue

        if id_of_row==1:
                # инициализируем количество визитов в магазин единицей
                n_visits=1

                # инициализируем строку для функции inizialize_row
                current_row = None

                # инициализируем количество визитов в магазин со скидкой
                n_red_visits=0

                # инициализируем стоимость отдельной позиции и количество товаров в чеке 
                sum_trn_sum_from_iss = 0
                sum_trn_sum_from_red = 0

                sum_product_quantity = 0
                sum_product_unique = 0

                # инициализируем листы фичей
                list_client_day = []                #дни посещений
                list_delay_days = []                #промужуток дней между посещений

                list_regular_points_received = []   #список полученных "регулярных"  баллов
                list_express_points_received = []   #список полученных "экспресс" баллов
                list_regular_points_spent = []      #список потраченных "регулярных"  баллов
                list_express_points_spent = []      #список потраченных "регулярных"  баллов

                list_purchase_sum = []              #список потраченных денег на покупки
                list_sum_product_quantity = []      #список числа купленных товаров
                list_sum_product_unique = []        #список числа уникальных купленных товаров

                list_trn_sum_from_iss = []          #список цен купленных позиций
                list_trn_sum_from_red = []          #список цен оплаченных бонусами
                list_price_from_iss = []            #список цен за одну единицу товара


        # Инициализируем текущую строку и предыдущую
        current_row, previos_row = inizialize_row(id_of_row, row, last_row=current_row)

        # Инициализируем текущие и прошлые значения id_клиента, транзакции, даты
        curr_client_id, prev_client_id = current_row[0], previos_row[0]
        curr_trans, prev_trans = current_row[1], previos_row[1]
        curr_date, prev_date = current_row[2], previos_row[2]

        # Если текущий чек продолжается
        if curr_trans==prev_trans:
            # Это тот же чек, того же клиента!

            #Обновляем признаки для текущего чека 
            # число уникальных товаров в чеке
            # число всех товаров в чеке
            # список цен за позицию для клиента по всем чекам   
            # список цен за уникальный товар, за позицию без скидки
            sum_product_unique,   \
            sum_product_quantity, \
            list_trn_sum_from_iss,\
            list_price_from_iss,  \
            list_trn_sum_from_red =  uptate_curr_check_feature(row,           
                                                              sum_product_unique,
                                                              sum_product_quantity, 
                                                              list_trn_sum_from_iss,
                                                              list_price_from_iss, 
                                                              list_trn_sum_from_red)

        # Если клиент тот же
        elif curr_client_id==prev_client_id:
            # Это новый чек, того же клиента! 

            list_of_last_visit= [list_client_day,
                                 list_regular_points_received,
                                 list_express_points_received,
                                 list_regular_points_spent,
                                 list_express_points_spent,

                                 list_purchase_sum,
                                 list_sum_product_quantity,
                                 list_sum_product_unique,

                                 sum_product_unique,
                                 sum_product_quantity
                                ]


            list_of_last_visit = summarize_last_visit(previos_row, prev_date, list_of_last_visit)

            sum_product_unique = list_of_last_visit[-2]
            sum_product_quantity = list_of_last_visit[-1]

            #Обновляем признаки для нового чека 
            # число уникальных товаров в чеке
            # число всех товаров в чеке
            # список цен за позицию для клиента по всем чекам   
            # список цен за уникальный товар, за позицию без скидки
            sum_product_unique,   \
            sum_product_quantity, \
            list_trn_sum_from_iss,\
            list_price_from_iss,  \
            list_trn_sum_from_red =  uptate_curr_check_feature(row,           
                                                              sum_product_unique,
                                                              sum_product_quantity, 
                                                              list_trn_sum_from_iss,
                                                              list_price_from_iss, 
                                                              list_trn_sum_from_red)

            # Число посещений +1
            n_visits+=1

            # Если нужно добавляем в число "красных" визитов 
            # Если нет нулей в списке, то покупка оплачена бонусами 
            if 0 not in list_trn_sum_from_red:
                n_red_visits+=1


        # Если клиент другой   curr_client_id!=prev_client_id
        elif curr_client_id!=prev_client_id:
            # Это другой клиент! 
            # Подводим итоги прошлого клиента!



            # Подводим итоги прошлого визита!
            list_of_last_visit= [list_client_day,
                                 list_regular_points_received,
                                 list_express_points_received,
                                 list_regular_points_spent,
                                 list_express_points_spent,

                                 list_purchase_sum,
                                 list_sum_product_quantity,
                                 list_sum_product_unique,

                                 sum_product_unique,
                                 sum_product_quantity
                                ]


            list_of_last_visit = summarize_last_visit(previos_row, prev_date, list_of_last_visit)
            sum_product_unique = list_of_last_visit[-2]
            sum_product_quantity = list_of_last_visit[-1]



            # Добавляем информацию о частоте посещения магазина

            list_delay_days, week_day_list = create_feat_from_days(list_client_day)

            # Собираем список из списков фичей - 11 штук !
            lists_of_feature = [list_delay_days, 
                                *list_of_last_visit[1:8], 
                                list_trn_sum_from_iss,
                                list_trn_sum_from_red,
                                list_price_from_iss
                               ]


            # Собираем итоговые признаки клиента
            # Инициалзируем лист который описывает клиента
            push=[]

            # Добавляем id, количество посещений, количество "красных" посещений
            push.extend([prev_client_id, n_visits, n_red_visits, *week_day_list])

            # Создаем фичи для каждого листа с помощью 
            # ряда функций  
            # list_func = [np.max, np.min, np.mean, np.median, np.std]
            for feature in create_feature(list_of_functions,lists_of_feature):
                push.append(feature)
            
            purchase.append(push)


            # Опустошаем использованные списки
            for item in lists_of_feature:
                item=[]

            n_visits = 1
            n_red_visits = 0

            list_delay_days=[]
            list_client_day=[]
            

        id_of_row += 1
        if id_of_row==nmax_rows:
            break
        
    return purchase

In [10]:
def generate_column_names(names_pure_feature=None, names_gen_feature=None, name_of_func=None):
    '''
    Generate names of columns for DataFrame.
    Takes names of pure features, names of generated features and names of functions.
    Return list of strings with column names.
    '''
    if names_pure_feature==None:
        names_pure_feature = ['client_id',
                              'n_visits',
                              'n_red_visits',
                              'weekday_1', 
                              'weekday_2',
                              'weekday_3',
                              'weekday_4',
                              'weekday_5',
                              'weekday_6',
                              'weekday_7',]
        
    if names_gen_feature==None:
        names_gen_feature = ['delay_days',       
                            'regular_points_received',
                            'express_points_received',
                            'regular_points_spent',
                            'express_points_spent',

                            'purchase_sum',
                            'sum_product_quantity',
                            'sum_product_unique',

                            'trn_sum_from_iss',
                            'trn_sum_from_red',
                            'price_from_iss'
                           ]
        
    if name_of_func==None:
        name_of_func = ['max', 'min', 'mean', 'median', 'std']
    
    

    columns=[]
    for name in names_pure_feature:
        columns.append(name)
    for feature in names_gen_feature:
            for func in name_of_func:
                columns.append(func + '_' + feature)
   
    return columns

columns = generate_column_names()

In [11]:
def create_dataframe(nmax_rows, filename='Data/purchases.csv'):
    '''
    Creates DataFrame from csv file with parcing maximum of nmax_rows.
    '''
    data = create_data_for_df(nmax_rows, filename)
    columns = generate_column_names()
    
    dataframe=pd.DataFrame(data=data, columns=columns)
    return dataframe

In [12]:
create_dataframe(4000)

,client_id,n_visits,n_red_visits,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,...,max_trn_sum_from_red,min_trn_sum_from_red,mean_trn_sum_from_red,median_trn_sum_from_red,std_trn_sum_from_red,max_price_from_iss,min_price_from_iss,mean_price_from_iss,median_price_from_iss,std_price_from_iss
0,000012768d,4,0,0,0,0,1,1,1,1,...,0.0,0.0,0.000000,0.0,0.000000,170.0,0.0,49.846154,45.5,29.106162
1,000036f903,32,0,3,7,6,3,3,1,9,...,0.0,0.0,0.000000,0.0,0.000000,367.0,0.0,54.824726,46.0,38.245587
2,000048b7a6,8,0,1,2,1,1,0,2,1,...,0.0,0.0,0.000000,0.0,0.000000,367.0,0.0,53.771144,44.5,42.485390
3,000073194a,17,0,2,0,1,4,3,5,2,...,35.0,0.0,0.375358,0.0,2.998485,370.0,0.0,61.310643,46.0,57.782340
4,00007c7133,11,0,2,1,1,1,1,2,3,...,250.0,0.0,1.584687,0.0,13.805465,674.0,0.0,62.121611,46.0,64.027768
5,00007f9014,29,0,1,2,3,2,10,6,5,...,366.0,0.0,2.290203,0.0,20.610885,674.0,0.0,61.717340,45.0,61.855139
6,0000a90cf7,35,0,4,4,5,8,4,4,6,...,402.0,0.0,3.857343,0.0,26.098347,674.0,0.0,63.475531,46.0,60.928998
7,0000b59cec,32,0,4,9,3,3,4,2,7,...,402.0,0.0,4.101818,0.0,26.274805,674.0,0.0,65.661824,50.0,61.186056
8,0000bb4e4e,7,0,0,0,2,0,2,3,0,...,402.0,0.0,4.320231,0.0,26.723294,674.0,0.0,65.708291,50.0,60.858200
9,0000bcec9c,56,0,11,9,7,12,7,5,5,...,402.0,0.0,3.444240,0.0,23.923840,674.0,0.0,65.633507,50.0,59.338715
